In [29]:
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import json
import matplotlib.pyplot as plt

In [30]:
volume_cols = ['siteID', 'richting', 'type', 'van', 'tot', 'aantal']

site_cols = ['siteID', 'sitenr', 'long', 'lat', 'naam', 'domein', 'wegnr', 'district', 'gemeente', 'interval', 'datum_van']

sitesdf = pd.read_csv('data/bike_BEL/sites.csv', names=site_cols)






In [31]:
df_cat = pd.merge(left = dfbikeBELjan, right = sitesdf[['siteID', 'lat', 'long']], left_on = 'siteID', right_on = 'siteID')

#df_group = df_cat.groupby([['lat', 'long']], dropna=False).mean()

df_group = df_cat.groupby(['siteID','lat', 'long']).sum().reset_index()
  


/tmp/ipykernel_5089/177028424.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_group = df_cat.groupby(['siteID','lat', 'long']).sum().reset_index()


In [32]:
#merge all the fietstelling palen into one dataframe and average the amount of cyclists per month per location
# importing libraries
import pandas as pd
import glob
import os


df_list = [] 

dir = 'data/bike_BEL/bikers_month/'

for csv in os.listdir(dir):
    csvpath = os.path.join(dir,csv)
    df = pd.read_csv(csvpath, names = volume_cols)
    df_list.append(df)

combo_df = pd.concat(df_list, axis=0)

df_cat2 = pd.merge(left = combo_df, right = sitesdf[['siteID', 'lat', 'long']], left_on = 'siteID', right_on = 'siteID')


df_group2 = df_cat2.groupby(['siteID','lat', 'long']).sum().reset_index()

df_group2['avg_month'] = df_group2['aantal'].apply(lambda x : x/(len(os.listdir(dir))))


df_group2.to_csv('avg_cyclists_month_LatLong.csv')

/tmp/ipykernel_5089/3689184235.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_group2 = df_cat2.groupby(['siteID','lat', 'long']).sum().reset_index()


In [3]:
#Try  the same for  the netherlands, download data from https://opendata.cbs.nl/statline/portal.html?_la=en&_catalog=CBS&tableId=84710ENG&_theme=1172


df_NL = pd.read_csv('data/bike_NL/84710ENG_UntypedDataSet_25032023_181036.csv', delimiter= ';')

#Then clean the columns a bit, Trips_4, DistanceTravelled_5, TimeTravelled_6 are trips, kilometers and time in hours per year per province
df_NL[['Trips_4', 'DistanceTravelled_5', 'TimeTravelled_6']] = df_NL[['Trips_4', 'DistanceTravelled_5', 'TimeTravelled_6']].replace('       .', 0).astype(float)


#provinces_names_dict = {'PV20' : 'Groningen','PV21' : 'Fryslân', 'PV22' : 'Drenthe', 'PV23' : 'Overijssel', 'PV24': 'Flevoland', 'PV25':'Gelderland', 'PV26' : 'Utrecht', 'PV27': 'Noord-Holland', 
#                  'PV28': 'Zuid-Holland', 'PV29':'Zeeland',  'PV30': 'Noord-Brabant', 'PV31': 'Limburg'}

#Change the characteristics column into their NUTS Level 2 designation
provinces_NUTS = {'PV20' : 'NL11','PV21' : 'NL12', 'PV22' : 'NL13', 'PV23' : 'NL21', 'PV24': 'NL23','PV25':'NL22', 'PV26' : 'NL31', 'PV27': 'NL32', 
                  'PV28': 'NL33', 'PV29':'NL34',  'PV30': 'NL41', 'PV31': 'NL42'}

df_NL['RegionCharacteristics'] = df_NL['RegionCharacteristics'].astype(str)
df_NL['RegionCharacteristics'] = df_NL['RegionCharacteristics'].replace(provinces_NUTS, regex=True)
df_NL= df_NL.rename(columns={'RegionCharacteristics':'NUTS_ID_LV2'})



In [9]:
#Take an average of each province's trip numbers, kilometers, and hours, to get an average 'speed' per propvince
#normalize this per province to get a sort of 'hinderance' factor
df_NL_region_year = df_NL.groupby('NUTS_ID_LV2').agg(mean_tripnum = ('Trips_4', 'mean'), mean_km = ('DistanceTravelled_5', 'mean'), mean_hours = ('TimeTravelled_6', 'mean'))

df_NL_region_year['avg_speed'] = df_NL_region_year[['mean_tripnum', 'mean_km', 'mean_hours']].apply(lambda x:  (x['mean_tripnum'] * x['mean_km']) / x['mean_hours'], axis=1)

df_NL_region_year.reset_index(inplace=True)

#apply Min-Max scaling to normalize the column   0 and 1
df_NL_region_year['normalized_avg_speed'] = (df_NL_region_year['avg_speed'] - df_NL_region_year['avg_speed'].min()) / (df_NL_region_year['avg_speed'].max() - df_NL_region_year['avg_speed'].min())

df_NL_region_year2 = df_NL_region_year.iloc[9:]


NameError: name 'datapackage' is not defined

In [ ]:
#Plot the NUTS geojson

nuts = gpd.read_file("data/NUTS_regions/NUTS_RG_01M_2021_4326.geojson")

#make for Benelux countries then NL
nuts_benelux = nuts.loc[nuts["CNTR_CODE"].isin(["BE","NL","LU"])]
nuts_nl = nuts_benelux.loc[nuts_benelux["CNTR_CODE"]=="NL"]

#trim the nuts_nl ID column to its first 4 characters, these are its NUTS_LV2 designation
nuts_nl['NUTS_ID_LV2'] = nuts_nl.apply(lambda row: row['NUTS_ID'][:4], axis=1)

#try to make a new column for normalized average speed on the nuts_Nl map to use on interactive map, unfortunately makes NaNs, and doesnt bring the data correctly
nuts_nl['norm_avg_speed'] = nuts_nl['NUTS_ID_LV2'].map(df_NL_region_year2.set_index('NUTS_ID_LV2')['avg_speed'])


nuts_nl

In [ ]:
from datapackage import Package
import pandas as pd

#if the merge would have worked, could have plotted the merged dataframe with teh polygons showing the 'hinderance' factor
data_url = 'https://datahub.io/core/geo-nuts-administrative-boundaries/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)
